In [ ]:
!git clone https://github.com/tiennguyen0000/CVDCLTGM.git

Cloning into 'CVDCLTGM'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 246 (delta 86), reused 131 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (246/246), 39.73 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
# !rm -r /content/WED
# !rm -r /content/CVDCLTGM
## update git repo

In [ ]:
!mv /content/CVDCLTGM/WED /content

In [ ]:
!pip install -r /content/WED/requirements.txt
!pip install -r /content/WED/requirementsAPI.txt
# if kaggle adjust content to kaggle/working

In [ ]:
!pip uninstall -y huggingface_hub
!pip install huggingface_hub==0.23.5

Found existing installation: huggingface-hub 0.26.2
Uninstalling huggingface-hub-0.26.2:
  Successfully uninstalled huggingface-hub-0.26.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
!pip install -U accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.25.0
    Uninstalling accelerate-0.25.0:
      Successfully uninstalled accelerate-0.25.0


In [ ]:
import torch

from WED.model.eunms import Model_Type, Scheduler_Type
from WED.model.Utils.enums_utils import get_pipes
from WED.model.config import RunConfig
from WED.schemas.genI_schemas import base64_to_image
from WED.model.main import run


def add_interrupt_attribute(self):
    self.interrupt = False  # Set to False by default

def crdeimg(scal, omega=3, t_exit=15):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model_type = Model_Type.SDXL
    scheduler_type = Scheduler_Type.DDIM
    pipe_inversion, pipe_inference = get_pipes(model_type, scheduler_type, device=device)
    input_image = base64_to_image(scal.img_base64)
    original_shape = input_image.size
    input_image = input_image.resize((1024, 1024))
    prompt = scal.promt # 'smile' for "009698.jpg", 'anime' for "Arknight.jpg"

    config = RunConfig(model_type = model_type,
                        num_inference_steps = 50,
                        num_inversion_steps = 50,
                        num_renoise_steps = 1,
                        scheduler_type = scheduler_type,
                        perform_noise_correction = False,
                        seed = 7865)
    
    _, inv_latent, _, all_latents, other_kwargs = run(input_image,
                                          prompt,
                                          config,
                                          pipe_inversion=pipe_inversion,
                                          pipe_inference=pipe_inference,
                                          do_reconstruction=False)


    pipe_inference.__class__.interrupt
    rec_image = pipe_inference(image = inv_latent,
                              prompt = scal.promt_fw,
                              denoising_start=0.0,
                              num_inference_steps = config.num_inference_steps,
                              guidance_scale = 1.0,
                                omega=omega, # omega=3 for "009698.jpg", omega=5 for "Arknight.jpg"
                                gamma=3, # gamma=3 for "009698.jpg", gamma=3 for "Arknight.jpg"
                                inv_latents=all_latents,
                                prompt_embeds_ref=other_kwargs[0],
                                added_cond_kwargs_ref=other_kwargs[1],
                                t_exit=t_exit, # t_exit=15 for "009698.jpg", t_exit=25 for "Arknight.jpg"
                                ).images[0]
    rec_image.resize(original_shape).save("new_sdxlcat_3.jpg")
    return rec_image

def genI(txt: TextInput):
    from WED.model.Pipeline.TTI import pipelineT2i, generator
    return pipelineT2i(txt.txt, generator=generator).images[0]

Keyword arguments {'safety_checker': None} are not expected by StableDiffusionXLDecompositionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
import torch
torch.cuda.empty_cache()
# if youo out memory, run this cell 

In [ ]:
import sys 
from pathlib import Path
# sys.path.append(str(Path(__file__).parent))
from WED.schemas.genI_schemas import tai_rps, TextInput, image_to_base64
sys.path.append(str(Path.cwd()))
from fastapi import FastAPI
import nest_asyncio
import uvicorn
import ngrok

app = FastAPI()

@app.post('/increase')
def predict(scal : tai_rps):
    response = crdeimg(scal, omega=3, t_exit=15)

    return image_to_base64(response)

@app.post('/decrease')
def predictcx(scal : tai_rps):
    response = crdeimg(scal, omega=-3, t_exit=15)
    # image = Image.open('/kaggle/input/dsgssdd/images.jpg')   
    return image_to_base64(response)
    # return 0

@app.post('/gen')
def predict(txt : TextInput):
    response = genI(txt)
    return image_to_base64(response)
    # return 0
    
@app.post('/test')
def predictcx(txt: TextInput):
    # fix bug
    return {'txt': txt.txt}

Inverting...


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
port = 8000
ngrok.set_auth_token("2oeQ1IbMhVUGy8RjTOtdXMX50cI_gQkjMK5ctG2L1Srdz1ZU")
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, port))
# for gpu onl
nest_asyncio.apply()
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=port)